In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew
%config Completer.use_jedi = False #for auto complete code 

In [ ]:
rng = pd.date_range('1981-01-01','2016-12-31', freq='d')
obs=pd.read_csv('D:/Proyectos_GitHub/TrmmCorrection/data/RAIN_GAUGE.csv')
obs_daily=obs.iloc[:,3:13152].transpose()
obs_daily=obs_daily.set_index(rng)

In [ ]:
plt.figure(figsize=(15,10))
sns.heatmap(obs_daily.isnull(), cbar=False)
plt.show()

In [ ]:
# Load precipitation data
# Assume the data has columns: 'Date', 'Precipitation'
data = pd.read_csv('precipitation_data.csv')

# Convert Date to datetime
data['Date'] = pd.to_datetime(data['Date'])

# Range Check
min_threshold = 0
max_threshold = 500
data['Range_Flag'] = data['Precipitation'].apply(lambda x: x < min_threshold or x > max_threshold)

# Temporal Consistency Check
data['Temp_Consistency_Flag'] = data['Precipitation'].diff().abs() > 100  # Example threshold

# Duplicate Records Check
data['Duplicate_Flag'] = data.duplicated(subset=['Date', 'Precipitation'])

# Zero Precipitation Check
# Example: Flag sequences of zeros longer than 10 days
data['Zero_Flag'] = (data['Precipitation'] == 0).astype(int)
data['Zero_Sequence'] = data['Zero_Flag'].groupby((data['Zero_Flag'] != data['Zero_Flag'].shift()).cumsum()).cumsum()
data['Zero_Sequence_Flag'] = data['Zero_Sequence'] > 10

# Extreme Value Check
upper_percentile = data['Precipitation'].quantile(0.99)
data['Extreme_Value_Flag'] = data['Precipitation'] > upper_percentile

# Summary of Flags
flag_columns = ['Range_Flag', 'Temp_Consistency_Flag', 'Duplicate_Flag', 'Zero_Sequence_Flag', 'Extreme_Value_Flag']
data['QC_Flag'] = data[flag_columns].any(axis=1)

# Print flagged data
print(data[data['QC_Flag']])
